In [1]:
# install packages if not installed
!pip install -q kagglehub
!pip install pytesseract

# import libraries
import kagglehub
import os
import pandas as pd
import numpy as np
import cv2 as cv
import json
import shutil
import pytesseract
import matplotlib.pyplot as plt
import spacy
import re
import numpy as np
from PIL import Image


In [2]:
# download dataset using kagglehub
path = kagglehub.dataset_download('senju14/ocr-dataset-of-multi-type-documents')

print(f"OCR Dataset Path: {path}")

# move dataset to /content for easy visibility
ocr_dataset = '/content/ocr_dataset'

shutil.copytree(path, ocr_dataset, dirs_exist_ok=True)

print("Datasets moved to /content/")

# list files in dataset
print("\nFiles in OCR Dataset:")

!ls /content/ocr_dataset


100%|██████████| 5.16G/5.16G [00:53<00:00, 104MB/s]

Extracting files...


OCR Dataset Path: /root/.cache/kagglehub/datasets/senju14/ocr-dataset-of-multi-type-documents/versions/3
Datasets moved to /content/

Files in OCR Dataset:
document  form	invoice  real_life


In [3]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
nlp = spacy.load("en_core_web_lg")

def extract_aadhar(image):

    # ocr from image
    text = pytesseract.image_to_string(image)

    # run spacy ner
    doc = nlp(text)

    # initialize fields
    name = dob = aadhar_number = address = None

    # extract entities from spacy ner
    for ent in doc.ents:
        label = ent.label_.upper()
        if label in ["NAME", "PERSON"] and name is None:
            name = ent.text.strip()
        elif label in ["DOB"] and dob is None:
            dob = ent.text.strip()
        elif label in ["AADHAAR", "AADHAR", "CARD_NO", "CARDNUMBER"] and aadhar_number is None:
            aadhar_number = ent.text.strip()
        elif label in ["ADDRESS", "LOC"] and address is None:
            address = ent.text.strip()

    # fallback using regex
    if not name:
        m = re.search(r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)?)', text)
        name = m.group(1) if m else None

    if not aadhar_number:
        match = re.search(r"\b\d{4}\s\d{4}\s\d{4}\b", text)
        aadhar_number = match.group(0) if match else None

    if not dob:
        match = re.search(r"\b\d{2}/\d{2}/\d{4}\b", text)
        dob = match.group(0) if match else None

    if not address:
      lines = [line.strip() for line in text.splitlines() if line.strip()]

      address_start = next((i for i, line in enumerate(lines) if re.search(r'(?i)\bAddress\b', line)), None)

      address_lines = []
      if address_start is not None:
          for line in lines[address_start + 1:]:
              address_lines.append(line)
              if re.search(r"\b\d{6}\b", line):
                  break
          address = " ".join(address_lines).strip() if address_lines else None
      else:
          address = None

    return {
        "Name": name,
        "DOB": dob,
        "Aadhar Number": aadhar_number,
        "Address": address,
    }

In [12]:
aadhar_path='/content/test2.jpg'
aadhar_img=Image.open(aadhar_path)
ocr_result=pytesseract.image_to_string(aadhar_img)
print(ocr_result)


 

  
 
 
    
     
 
 
    
 
  

HNA OAT
Government of India
verre aidrde wets
Prakash Gopichand Rathod
sen artra / DOB : 01/07/1974
get / Male

Unique Identification Authority of India

Ocal Address:

S/O: aitrag wels, wa - 14, S/O: Gopichand Rathod, Sr

ars a- 3, Waa, Sena Fictat No-14, Ward No-3, Ramnagar,

Us, arefaara Het Haw, Arasr, Deccan Collage Road, Near

or wet, Aas, 7, AEN, Balvikas Kendra, Yerwada, Pune

411006 City, Yerwada, Pune,
Maharashtra, 411006

  
 

8538 7535 4900
x

1947
1800 300 1947 help @uldai.gov.in www.uidai.gov.

  

 



In [13]:
from PIL import Image
from IPython.display import JSON, display

data = extract_aadhar(aadhar_img)

display(JSON(data))

with open('aadhaar_extraction.json', 'w') as f:
    json.dump(data, f, indent=2)

<IPython.core.display.JSON object>